In [ ]:
peft_model_repo_id = "aryaniyaps/finetuned_flan_t5_xl_lease_data_extraction_200"

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
 
# Load the Lora model
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained"./saved_models/finetuned_flan_t5_xl_lease_data_extraction_200", device_map={"":0})

In [ ]:
finetuned_model.push_to_hub(peft_model_repo_id)